In [2]:
from IO.config import get_usr_config
from IO.result import DetectionResult
from core import detector
from core import freq_transform
from core import signal_generator
from core import utils
from core.utils import MICROSECONDS_IN_SECONDS
from IO import config

import matplotlib.pyplot as plt
import numpy as np
import torch

from datetime import datetime

In [34]:
class NoiseModelData:
    def __init__(
        self, 
        dim,
        Bx, Bw, Ba,
        noise_level_db,
        model_coeff_generator,
        
        deltax, deltaw, deltay,
        var_weight,
        noise_level_linear,
        
        qoutput,
        noise_to_out,
        qinput_to_out,
        qweight_to_out_by_signal,
        qweight_to_out_by_noise
    ):
        self.dim = dim
        self.Bx = Bx
        self.Bw = Bw
        self.Ba = Ba
        self.noise_level_db = noise_level_db
        self.model_coeff_generator = model_coeff_generator
        self.deltax = delta
        self.deltaw = deltaw
        self.deltay = deltay
        self.var_weight = var_weight
        self.noise_level_linear = noise_level_linear
        self.noise_to_out = noise_to_out
        self.qoutput = qoutput
        self.noise_to_out = noise_to_out,
        self.qinput_to_out = qinput_to_out
        self.qweight_to_out_by_signal = qweight_to_out_by_signal
        self.qweight_to_out_by_noise = qweight_to_out_by_noise

In [52]:
def unisign_quant(data, n_bits, clip, quant_flag=False):
    if not quant_flag:
        return data
    
    data = torch.Tensor(data)
    w_c = data.clamp(-clip, clip)
    b = torch.pow(torch.tensor(2.0), 1 - n_bits)
    w_q = clip * torch.min(b * torch.round(w_c / (b * clip)), 1 - b)

    return w_q.numpy()


In [56]:
weight_quant = True
acc_quant = False
input_quant = False

for noise_level in range(-30,11):
    for b in range(2, 11):
        configurations = config.parse_config('../yaml/example.yaml')

        noise_level_db = noise_level
        noise_level_linear = utils.db_to_linear(noise_level_db)

        configurations.signal.num_pos_decision = 1000
        configurations.signal.amps = [0.365]
        configurations.noise.init_args.top = noise_level_linear
        configurations.noise.init_args.steady_state = noise_level_linear

        generator = signal_generator.InputSignalGenerator(configurations.signal, configurations.noise)
        signal_with_noise, _ = generator.get()

        configurations.noise.init_args.top = 0
        configurations.noise.init_args.steady_state = 0

        generator = signal_generator.InputSignalGenerator(configurations.signal, configurations.noise)
        signal_without_noise, _ = generator.get()

        noise_sample = signal_with_noise - signal_without_noise

        Nd = configurations.signal.num_pos_decision
        N = configurations.signal.block_size
        signal_with_noise = signal_with_noise[0:Nd,:,:]
        signal_without_noise = signal_without_noise[0:Nd,:,:]
        noise_sample = noise_sample[0:Nd,:,:]

        coeff_jdht = np.zeros((Nd,N,N))
        for i in range(Nd):
            coeff_jdht[i, :, :] = freq_transform.dht_coeff_jitter(N)/N

        var_jdht_coeff = np.var(coeff_jdht)
        signal_power = np.var(signal_without_noise)

        Bw = b
        w_max = np.abs(coeff_jdht).max()
        qcoeff_jdht = unisign_quant(coeff_jdht, Bw, w_max, weight_quant)
        deltaw = utils.get_delta(Bw, w_max)

        Bx = b
        sigal_max = np.abs(signal_with_noise).max()
        qsignal_with_noise =  unisign_quant(signal_with_noise, Bx, sigal_max, input_quant)
        deltax = utils.get_delta(Bx, sigal_max)

        result_noise = np.zeros(signal_without_noise.shape)
        result_signal_without_noise = np.zeros(signal_without_noise.shape)
        result_signal_with_noise = np.zeros(signal_without_noise.shape)
        qresult_signal_without_noise = np.zeros(signal_without_noise.shape)
        qresult_signal_with_noise = np.zeros(signal_without_noise.shape)

        start = datetime.now()
        for i in range(Nd):
            result_signal_without_noise[i,:,:] = signal_without_noise[i,:,:].dot(coeff_jdht[i,:,:].T)
            result_noise[i,:,:]  = noise_sample[i,:,:].dot(coeff_jdht[i,:,:].T)
            result_signal_with_noise[i, : ,:] = signal_with_noise[i,:,:].dot(coeff_jdht[i,:,:].T)
            
            qresult_signal_with_noise[i, :, :] = qsignal_with_noise[i,:,:].dot(qcoeff_jdht[i,:,:].T)

        Ba = b
        result_max = np.abs(result_signal_with_noise).max()
        deltay = utils.get_delta(Ba, result_max)
        qresult_signal_with_noise = unisign_quant(qresult_signal_with_noise, Ba, result_max, acc_quant)
        end = datetime.now()

        total_dp_noise = qresult_signal_with_noise - result_signal_without_noise
        var_total_dp_noise = total_dp_noise.var()
        A = deltay**2/12 if acc_quant else 0
        B = N * var_jdht_coeff * noise_level_linear 
        C = N/12 * var_jdht_coeff * deltax**2 if input_quant else 0
        D = N/12 * signal_without_noise.var() * deltaw**2 if weight_quant else 0
        E = N/12 * noise_level_linear * deltaw**2 if weight_quant else 0
        
        predicted_signal_power = N * signal_power * var_jdht_coeff
        sim_signal_power = result_signal_without_noise.var()
        
        predicted_snr = predicted_signal_power / (A+B+C+D+E)
        sim_snr = sim_signal_power / var_total_dp_noise
        print('total time (s):', utils.format_float((end-start).total_seconds()), 'B:', b,
              noise_level, 'dB:', 10*np.log10((A+B+C+D+E)/var_total_dp_noise), 
             10*np.log10(predicted_signal_power/sim_signal_power),
             10*np.log10(predicted_snr/sim_snr))

total time (s):  0.008 B: 2 -30 dB: -2.1440539066370916 0.10735614097122023 2.251410047608312
total time (s):  0.008 B: 3 -30 dB: -1.0367462906323 0.05613986456384616 1.0928861551961453
total time (s):  0.008 B: 4 -30 dB: -0.2839891968052387 -0.0012723974106489452 0.28271679939459
total time (s):  0.008 B: 5 -30 dB: 0.053240875980445315 0.02148623272001786 -0.031754643260427345
total time (s):  0.010 B: 6 -30 dB: -0.03173170111977241 -0.07181161493589275 -0.04007991381612118
total time (s):  0.008 B: 7 -30 dB: -0.14631784003515347 -0.025374565032194235 0.12094327500295883
total time (s):  0.008 B: 8 -30 dB: -0.17568085539569042 -0.056057667739779396 0.11962318765591139
total time (s):  0.008 B: 9 -30 dB: 0.10784256187162825 -0.06934367718605272 -0.177186239057681
total time (s):  0.009 B: 10 -30 dB: -0.031635246969437786 0.01827982391100778 0.049915070880446576
total time (s):  0.008 B: 2 -29 dB: -2.0290250281305786 0.000784495054236602 2.0298095231848152
total time (s):  0.009 B: 3 -2

total time (s):  0.010 B: 5 -21 dB: 0.12110568420768578 0.018684577571217515 -0.10242110663646853
total time (s):  0.010 B: 6 -21 dB: -0.060846514192522586 -0.021139332344209276 0.03970718184831408
total time (s):  0.010 B: 7 -21 dB: 0.02929128799572599 -0.029015738689374945 -0.05830702668510067
total time (s):  0.011 B: 8 -21 dB: 0.06811371902299772 -0.06395492319165386 -0.13206864221465175
total time (s):  0.010 B: 9 -21 dB: -0.06384009882119664 0.018173895136871954 0.08201399395806809
total time (s):  0.010 B: 10 -21 dB: 0.06709731585634952 0.028840180589069016 -0.03825713526728069
total time (s):  0.008 B: 2 -20 dB: 0.06412575675578844 -0.03647272487279478 -0.10059848162858287
total time (s):  0.008 B: 3 -20 dB: 0.1969024654471371 0.023435977634088594 -0.17346648781304844
total time (s):  0.010 B: 4 -20 dB: 0.20313110698868872 0.026331899491352638 -0.17679920749733555
total time (s):  0.008 B: 5 -20 dB: 0.044323139184924004 -0.037485849394440135 -0.08180898857936401
total time (s):

total time (s):  0.009 B: 8 -12 dB: 0.043301670374704566 0.04211518520221297 -0.0011864851724920228
total time (s):  0.008 B: 9 -12 dB: 0.024861150242004772 -0.0073401504034981396 -0.032201300645503306
total time (s):  0.009 B: 10 -12 dB: -0.06427022803598896 0.0014020461032926675 0.06567227413928083
total time (s):  0.008 B: 2 -11 dB: 1.0248747451099918 0.008965341652851866 -1.0159094034571399
total time (s):  0.009 B: 3 -11 dB: 0.4622579598790672 0.04821918490619467 -0.41403877497287167
total time (s):  0.013 B: 4 -11 dB: 0.19462157166576455 0.028322878749516903 -0.16629869291624774
total time (s):  0.008 B: 5 -11 dB: 0.02300948352759428 0.029184184205881366 0.006174700678286686
total time (s):  0.008 B: 6 -11 dB: 0.08371209267811777 -0.017809405431376275 -0.1015214981094939
total time (s):  0.009 B: 7 -11 dB: 0.02986739977960761 -0.03509482122840706 -0.06496222100801459
total time (s):  0.009 B: 8 -11 dB: 0.020633741489287656 0.03054066388425361 0.009906922394966634
total time (s): 

total time (s):  0.008 B: 4 -2 dB: 0.24629742664589188 -0.09014968310006549 -0.33644710974595754
total time (s):  0.008 B: 5 -2 dB: 0.114283101634676 0.03373571068397188 -0.08054739095070473
total time (s):  0.010 B: 6 -2 dB: 0.014211143153682507 -0.027907204518498728 -0.04211834767218185
total time (s):  0.008 B: 7 -2 dB: -0.05822645414536094 -0.025580999200787817 0.03264545494457283
total time (s):  0.009 B: 8 -2 dB: -0.09940914606903803 -0.025228188713375142 0.07418095735566238
total time (s):  0.008 B: 9 -2 dB: -0.013640540840382406 0.014353591547271623 0.02799413238765372
total time (s):  0.009 B: 10 -2 dB: -0.020696704035798662 -0.07406595552028315 -0.05336925148448427
total time (s):  0.008 B: 2 -1 dB: 1.2696085542913704 -0.07060478998188048 -1.3402133442732507
total time (s):  0.009 B: 3 -1 dB: 0.483349951348181 -0.08067153206895743 -0.5640214834171379
total time (s):  0.008 B: 4 -1 dB: 0.18963740682561375 -0.054174479944412116 -0.2438118867700257
total time (s):  0.009 B: 5 -1

total time (s):  0.009 B: 9 7 dB: 0.0111378974089399 0.06153584249984918 0.05039794509090907
total time (s):  0.008 B: 10 7 dB: 0.017567282936094174 0.059483370955036474 0.04191608801894278
total time (s):  0.008 B: 2 8 dB: 1.2912614505476823 0.05670101528987475 -1.2345604352578075
total time (s):  0.008 B: 3 8 dB: 0.46819709127874415 0.006265559543979216 -0.46193153173476464
total time (s):  0.008 B: 4 8 dB: 0.18692584277147137 0.0037280564013726695 -0.18319778637009929
total time (s):  0.008 B: 5 8 dB: 0.08941719135198428 0.09005884776911077 0.0006416564171266211
total time (s):  0.008 B: 6 8 dB: -0.005120791970130019 -0.00394773934377177 0.0011730526263580197
total time (s):  0.009 B: 7 8 dB: 0.040761515941862544 -0.03150424658611617 -0.07226576252797856
total time (s):  0.008 B: 8 8 dB: -0.017413333189218665 -0.019771010770032946 -0.0023576775808142616
total time (s):  0.009 B: 9 8 dB: -0.018912782575701233 0.02564804247227597 0.044560825047977065
total time (s):  0.008 B: 10 8 dB: